<a href="https://colab.research.google.com/github/GwiHwan-Go/Detect_SC/blob/main/Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 수집한 데이터를 분석해 봅시다.

##### 본 스크립트의 코드들은 대부분 1-2시간 정도의 연산 시간을 필요로 합니다.
##### 실질적인 데이터 분석 및 시각화는 Data_Visualization 스크립트를 참고해주세요.

참고 자료 : [딥러닝을 이용한 자연어 처리 입문](!https://wikidocs.net/92961), [SoyNLP 공식 문서](!https://github.com/lovit/soynlp)

# 준비 사항

본 파일은 체험용 코드입니다.

실제로 본 파일에 사용된 데이터는 240만 개의 게시글이며, 

여기서는 간단한 체험을 위해서 체험용 데이터를 제공합니다. 

또한, 미리 훈련시킨 soyNLP Tokenizer 를 제공하며, 

다운로드 방법은 아래에서 설명할 것입니다.

(수정중)

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'Detect_SC_symptoms'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/MyDrive/{}'.format(FOLDERNAME))

# Change dariectory to current folder
%cd /content/drive/MyDrive/$FOLDERNAME

Mounted at /content/drive
/content/drive/MyDrive/Detect_SC_symptoms


In [ ]:
!pip install soynlp
!pip install konlpy
import pandas as pd

In [ ]:
# 결과물 저장 경로
save_dir = "/content/drive/MyDrive/Detect_SC_symptoms/results"
file_name = "refined_full_custom_tokenizer.csv" # tokenized X = refined_full.csv / with Okt tokenizer = refined_full_okt_tokenized.csv
data_path = f"/content/drive/MyDrive/Detect_SC_symptoms/results/{file_name}" 
data = pd.read_csv(data_path, low_memory=False, index_col=0) 
first_id, last_id = data.loc[0,'id'], data.loc[data.shape[0]-1,'id']
print("수집한 게시물 범위 :", first_id, "~", last_id, "\n" \
      "수집된 데이터 크기 :", data.shape[0], "\n수집한 데이터 features :", data.columns)
data.tail()

수집한 게시물 범위 : 2 ~ 4445728 
수집된 데이터 크기 : 2470236 
수집한 데이터 features : Index(['id', 'title', 'author', 'date', 'link'], dtype='object')


,id,title,author,date,link
2470231,4445724,나 병신 ...,민애,2022-10-27,https://m.dcinside.com/board/depression_new1/4...
2470232,4445725,나고 닉 글 올리 규 싶 읃데,조팡이(118.235),2022-10-27,https://m.dcinside.com/board/depression_new1/4...
2470233,4445726,씨발 남자 직원 음식 깔아 주,연우ㅗ(223.62),2022-10-27,https://m.dcinside.com/board/depression_new1/4...
2470234,4445727,나도 근데 아이디 두 번 바꿈,.O2,2022-10-27,https://m.dcinside.com/board/depression_new1/4...
2470235,4445728,알바 바보 쉨 ㅋㅋㅋ 나도 차단 해봐라,민애,2022-10-27,https://m.dcinside.com/board/depression_new1/4...


In [ ]:
print(f"중복, NAN 제거 전 데이터 수 = {data.shape}")
data = data.drop_duplicates(['id'])
data = data[data.title.isna()==False]
data = data.reset_index(drop=True)
print(f"중복, NAN 제거 후 데이터 수 = {data.shape}")

중복, NAN 제거 전 데이터 수 = (2470236, 5)
중복, NAN 제거 후 데이터 수 = (2470236, 5)


### 게시글을 전처리 해보자.

1. Hanspell -> 신조어에 대응을 아예 못함
2. SoyNLP 를 이용해, 모은 자료들을 통해서 학습시킴.

   -> 신조어나 형태소 분석기에 등록되지 않은 단어를 잘 구분해줄 수 있음.

In [ ]:
# Hanspell, koNLPY 예제
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install konlpy

from hanspell import spell_checker

for sent in data.title[-20:] :
  okt = Okt()
  print('before :', sent)
  spelled_sent = spell_checker.check(sent)

  hanspell_sent = spelled_sent.checked
  print('after :', hanspell_sent)
  print('after :', okt.pos(sent, stem=True))

In [ ]:
from soynlp.word import WordExtractor

model_fname="/content/drive/MyDrive/Detect_SC_symptoms/models" + "/" + "soy_wordExtractor_full.model"

##SoyNLP 훈련 및 저장 코드 - 30분 걸림##

# word_extractor = WordExtractor()
# word_extractor.train(data[data.title.isna()==False].title)
# word_score_table = word_extractor.extract()
# word_extractor.save(model_fname)

##SoyNLP 훈련 및 저장 코드 - 30분 걸림##

training was done. used memory 6.691 Gb
all cohesion probabilities was computed. # words = 480337
all branching entropies was computed # words = 523918
all accessor variety was computed # words = 523918


In [ ]:
import math
from soynlp.tokenizer import LTokenizer

## Word_Extractor param 설정 및 로드
word_extractor = WordExtractor(min_frequency=10,
                               min_cohesion_forward=5.0,
                               min_right_branching_entropy=0.0)
word_extractor.load(model_fname)

## Cohesion 과 Entropy 계산해서 tokenizer 에 필요한 scores 계산
scores = word_extractor.word_scores()
scores = {key : (scores[key].cohesion_forward * math.exp(scores[key].right_branching_entropy)) \
          for key in scores.keys()}

## tokenizer 생성
tokenizer = LTokenizer(scores=scores)

all cohesion probabilities was computed. # words = 480337
all branching entropies was computed # words = 523918
all accessor variety was computed # words = 523918


In [ ]:
## SoyNLP 사용 예 -> SoyNLP(tokenizer) -> okt pos tagger -> 불용어 제거
from konlpy.tag import Okt
from soynlp.normalizer import *


def preprocess_tokenizer(sentence) : 

  pos_tagger = Okt()
  # sentence = emoticon_normalize(sentence)
  # sentence = repeat_normalize(sentence)

  #soynlp tokenizer로 나눠준 후 konlpy Okt postagger로 품사 판별
  return [word[0]
          for word in pos_tagger.pos(' '.join(tokenizer.tokenize(sentence)))
          if word[1] != 'Josa'] # 조사는 분석에서 제외

#### 240m 건의 텍스트 데이터를 정제(토큰화 + 조사 제거)하는 작업 - 1시간 걸림 ####
# from tqdm import tqdm
# tqdm.pandas()
# data.title = data.title.progress_apply(lambda x : ' '.join(preprocess_tokenizer(x)))  OR # data.title = data.title.progress_apply(lambda x : ' '.join(okt.morphs(x)))

100%|██████████| 2470236/2470236 [57:28<00:00, 716.24it/s]


## PMI 분석


단어들 간의 연관관계를 분석해서 자살 관련 정보 파악에 도움이 되는 __다른 단어__들은 무엇이 있는 지 알아보자.


In [ ]:
from soynlp.vectorizer import sent_to_word_contexts_matrix
from scipy import sparse
import pickle
from soynlp.word import pmi as pmi_func

mat_save_path = "/content/drive/MyDrive/Detect_SC_symptoms/models/words_sparse_mat.npz"
index_list_save_path = "/content/drive/MyDrive/Detect_SC_symptoms/models/idx_to_vocabs"

############ 텍스트 데이터를 정제,변환해서 주위 단어들과 함께 word matrix로 변환해주는 작업 - 2시간 30분 정도 걸림 ########
# x, idx2vocab = sent_to_word_contexts_matrix(
#     data.title,
#     windows=3,
#     min_tf=10,
#     tokenizer= lambda x : okt.morphs(x), # (default) lambda x:x.split(),
#     dynamic_weight=False,
#     verbose=True
# )

# sparse.save_npz(mat_save_path, x)
# with open(index_list_save_path, "wb") as fp:   #Pickling
#   pickle.dump(idx2vocab, fp)
############ 텍스트 데이터를 정제,변환해서 주위 단어들과 함께 word matrix로 변환해주는 작업 - 2시간 30분 정도 걸림 ########

###Load Data

x = sparse.load_npz(mat_save_path)
with open(index_list_save_path, "rb") as fp:   # Unpickling
  idx2vocab = pickle.load(fp)

pmi, px, py = pmi_func(
    x,
    min_pmi = 0,
    alpha = 0.0,
    beta = 0.75
)

In [ ]:
## 특정 단어['살고 싶다']와 함께 등장한 단어들 상위 10개##
target = '살고싶다'
howmany = 10

vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
query = vocab2idx[target]

submatrix = pmi[query,:].tocsr() # get the row of query
contexts = submatrix.nonzero()[1] # nonzero() return (rows, columns)
pmi_i = submatrix.data

most_relateds = [(idx, pmi_ij) for idx, pmi_ij in zip(contexts, pmi_i)]
most_relateds = sorted(most_relateds, key=lambda x:-x[1])[:howmany]
most_relateds = [(idx2vocab[idx], pmi_ij) for idx, pmi_ij in most_relateds]

from pprint import pprint
pprint(most_relateds)

[('똑똑하게', 5.852626621112024),
 ('따라가서', 5.5436417792650055),
 ('이글루', 5.509046737995528),
 ('베어그릴스', 5.420709461253241),
 ('말안하고', 5.3476182061642),
 ('ㄹㅇㄹㅇㄹㅇ', 5.287448578325152),
 ('배당금', 5.287448578325152),
 ('사령관', 5.2702064396566275),
 ('평화롭게', 5.236867617728502),
 ('나아서', 5.204947906914405)]


In [ ]:
## 특정 단어['우울']과 컨텍스트가 가장 비슷한 단어 상위 10개##

from soynlp.utils import most_similar

most_similar('우울', pmi, vocab2idx, idx2vocab)

[('심심', 0.22781712789543918),
 ('불안', 0.20654552217405342),
 ('우울함', 0.16989673506620928),
 ('행복', 0.16271023927659378),
 ('우울증', 0.1625480654125695),
 ('피곤', 0.1547978316176476),
 ('우울해', 0.14304488688384553),
 ('우울하다', 0.13556653025904541),
 ('불행', 0.13496097718192268),
 ('우울한', 0.13324270584997489)]

# 텍스트 분석 끝!
